In [78]:
import os

# Set working directory to your project root explicitly
os.chdir(r"C:\Users\user\Python Programs\Resume Projects\End-to-end-Wine-quality")

print("✅ Current directory:", os.getcwd())


✅ Current directory: C:\Users\user\Python Programs\Resume Projects\End-to-end-Wine-quality


In [79]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [80]:
from mlProject.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from mlProject.utils.common import read_yaml,crate_directories

In [81]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        crate_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        crate_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(root_dir=config.root_dir,source_URL=config.source_URL,local_data_file=config.local_data_file,unzip_dir=config.unzip_dir)
        return data_ingestion_config


In [82]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [83]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,header=request.urlretrieve(url=self.config.source_URL,filename=self.config.local_data_file)
            logger.info(f"{filename} download with following info :\n{header}")
        else:
            logger.info(f"File already exists of size:{get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
            zip_file_path:str
            Extract the zip file into the data directory
            Function returns None
        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [84]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-20 23:13:30,496: INFO: common: yaml file:config\config.yaml loaded sucessfully]
[2025-06-20 23:13:30,500: INFO: common: yaml file:params.yaml loaded sucessfully]
[2025-06-20 23:13:30,503: INFO: common: yaml file:schema.yaml loaded sucessfully]
[2025-06-20 23:13:30,505: INFO: common: created directory at:artifacts]
[2025-06-20 23:13:30,507: INFO: common: created directory at:artifacts/data_ingestion]
[2025-06-20 23:13:32,364: INFO: 2231650630: artifacts/data_ingestion/data.zip download with following info :
Connection: close
Content-Length: 26176
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f751ec0917e15a3dc07c3094a49fb99713109dd8a9f29150bb5d187ec17facdf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D404:2765E9:413CA:9F43B:68559DC4
Accept-Ranges: bytes
Date: Fri, 20 Jun 2025 1